<a href="https://colab.research.google.com/github/solaimanhridoy/beam-dynamics-task-qa-challenge/blob/main/beamdynamics_QA_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Load Excel file and skip first row
df = pd.read_excel('/content/drive/MyDrive/Data/Task+File+-+Data+Quality+Analyst.xlsx', skiprows=1, header=None)

# Set second row as column names
df.columns = df.iloc[0]

# Drop the second row (previously the first row)
df = df.drop(0)

In [ ]:
df.head()

,Don't modify this row,Manufacturer,Model #,Product Name,Product Page URL,Manual or Brochure URL,Latest Firmware Update URL,Weight [in lb only]
1,Example (Please don't modify this row),Sony,ILCE7CL/B,a7C Mirrorless Camera,https://www.bhphotovideo.com/c/product/1592779...,https://www.bhphotovideo.com/lit_files/685187.pdf,https://support.d-imaging.sony.co.jp/app/uud/m...,1.1
2,You should find value for this product from In...,Aida,UHD6G-200,Professional EFP Camera,NaN,NaN,NaN,NaN
3,You should find value for this product from In...,Fujifilm,16781591,Mirrorless Digital Camera,NaN,NaN,NaN,NaN
4,You should find value for this product from In...,Angelbird,AVP1TBCF,AV PRO CF 1TB CFast 2.0,NaN,NaN,NaN,NaN
5,You should find value for this product from In...,Anton Bauer,8675-0164,Titon Micro 90 Gold Mount,NaN,NaN,NaN,NaN


## Search Product Page URL

In [ ]:
import pandas as pd
from googlesearch import search
import time

# create a new dataframe to store search results
search_results = pd.DataFrame(columns=["Manufacturer", "Model #", "Product Name", "B&H URL"])

# iterate over each row in the dataframe and search for the URL
for i, row in df.iterrows():
    # define the search term
    search_term = f"{row['Manufacturer']} {row['Model #']} {row['Product Name']} B&H"
    
    # search for the URL on B&H and store the result in the search_results dataframe
    for url in search(search_term, stop=1):
        search_results.loc[i] = [row['Manufacturer'], row['Model #'], row['Product Name'], url]
        break
    else:
        search_results.loc[i] = [row['Manufacturer'], row['Model #'], row['Product Name'], "NA"]
        
    time.sleep(5)

# Add suffix "_search_results" to the column names of the search_results data frame
search_results = search_results.add_suffix("_search_results")

# Merge the two data frames based on their index
merged_df = pd.concat([df, search_results], axis=1)

# save the results to a CSV file
df.to_csv("search_results.csv", index=False)


## Search Manual or Brochure Page URL

In [ ]:
# create a new column for manual/brochure URL
df["Manual/Brochure URL"] = "NA"

# iterate over the rows and search for the user manual or brochure URL
for index, row in df.iterrows():
    search_terms = [row["Manufacturer"] + " " + row["Model #"] + " manual site:bhphotovideo.com",
                    row["Manufacturer"] + " " + row["Model #"] + " brochure site:bhphotovideo.com",
                    row["Manufacturer"] + " " + row["Model #"] + " manual site:adorama.com",
                    row["Manufacturer"] + " " + row["Model #"] + " brochure site:adorama.com",
                    row["Manufacturer"] + " " + row["Model #"] + " manual",
                    row["Manufacturer"] + " " + row["Model #"] + " brochure"]

    for search_term in search_terms:
        for url in search(search_term, stop=1):
            if "bhphotovideo.com" in url or "adorama.com" in url or row["Manufacturer"].lower() in url:
                df.at[index, "Manual/Brochure URL"] = url
                break
        if df.at[index, "Manual/Brochure URL"] != "NA":
            break

    time.sleep(5)

# Add suffix "_search_results" to the column names of the search_results data frame
search_results = search_results.add_suffix("_search_results")

# Merge the two data frames based on their index
merged_df = pd.concat([df, search_results], axis=1)

# # save the results to a CSV file
# df.to_csv("search_results.csv", index=False)


## Search Firmware/Software URL

In [ ]:
import pandas as pd
import time
from googlesearch import search
import re

# create an empty dataframe to store the search results
search_results = pd.DataFrame(columns=["Manufacturer", "Model #", "Product Name", "Product Page URL", "Firmware/Software Update URL"])

# create a list of search terms to use for the Google search
search_terms = []
for i in range(len(df)):
    search_terms.append(df.loc[i, "Manufacturer"] + " " + df.loc[i, "Model #"] + " firmware/software update download")

# search for the URLs on B&H and store the results in the search_results dataframe
for i, search_term in enumerate(search_terms):
    for url in search(search_term, stop=1):
        search_results.loc[i, "Manufacturer"] = df.loc[i, "Manufacturer"]
        search_results.loc[i, "Model #"] = df.loc[i, "Model #"]
        search_results.loc[i, "Product Name"] = df.loc[i, "Product Name"]
        search_results.loc[i, "Product Page URL"] = url
        
        # search for firmware/software update URLs
        firmware_search_terms = ["firmware download", "software update download", "firmware/software update download"]
        firmware_urls = []
        for firmware_search_term in firmware_search_terms:
            firmware_search_query = df.loc[i, "Manufacturer"] + " " + df.loc[i, "Model #"] + " " + firmware_search_term
            for firmware_url in search(firmware_search_query, stop=1):
                if "bhphotovideo.com" in firmware_url or "adorama.com" in firmware_url:
                    if "windows" in firmware_url and firmware_url.endswith(".zip"):
                        firmware_urls.append(firmware_url)
                    elif "windows" in firmware_url:
                        firmware_urls.append(firmware_url)
                    elif firmware_url.endswith(".zip"):
                        firmware_urls.append(firmware_url)
                    else:
                        firmware_urls.append(firmware_url)
        if firmware_urls:
            search_results.loc[i, "Firmware/Software Update URL"] = firmware_urls[0]
        else:
            search_results.loc[i, "Firmware/Software Update URL"] = "NA"
        break
    else:
        search_results.loc[i, "Manufacturer"] = df.loc[i, "Manufacturer"]
        search_results.loc[i, "Model #"] = df.loc[i, "Model #"]
        search_results.loc[i, "Product Name"] = df.loc[i, "Product Name"]
        search_results.loc[i, "Product Page URL"] = "NA"
        search_results.loc[i, "Firmware/Software Update URL"] = "NA"
    time.sleep(5)

# output the search results to a CSV file
search_results.to_csv("search_results.csv", index=False)


## Search Weight 

In [ ]:
df.head()

,Don't modify this row,Manufacturer,Model #,Product Name,Product Page URL,Manual or Brochure URL,Latest Firmware Update URL,Weight [in lb only],B&H URL,Adorama URL
1,Example (Please don't modify this row),Sony,ILCE7CL/B,a7C Mirrorless Camera,https://www.bhphotovideo.com/c/product/1592779...,https://www.bhphotovideo.com/lit_files/685187.pdf,https://support.d-imaging.sony.co.jp/app/uud/m...,NA,https://www.bhphotovideo.com/c/product/1592776...,https://www.adorama.com/isoa7cb.html
2,You should find value for this product from In...,Aida,UHD6G-200,Professional EFP Camera,NaN,NaN,NaN,NA,https://www.bhphotovideo.com/c/product/1382661...,https://www.adorama.com/aiuhd6g200.html
3,You should find value for this product from In...,Fujifilm,16781591,Mirrorless Digital Camera,NaN,NaN,NaN,NA,https://www.bhphotovideo.com/c/product/1726020...,https://www.adorama.com/ifjxh2k.html
4,You should find value for this product from In...,Angelbird,AVP1TBCF,AV PRO CF 1TB CFast 2.0,NaN,NaN,NaN,NA,https://www.bhphotovideo.com/c/product/1452851...,https://www.adorama.com/abavp1tbcf.html
5,You should find value for this product from In...,Anton Bauer,8675-0164,Titon Micro 90 Gold Mount,NaN,NaN,NaN,NA,https://www.bhphotovideo.com/c/product/1596814...,https://www.adorama.com/ab86750164.html


In [ ]:
from googlesearch import search
import pandas as pd
import time
import requests
import re

search_results = pd.DataFrame(columns=["Manufacturer", "Model #", "Product Name", "Weight [in lb only]"])

for i, row in df.iterrows():
    search_terms = [row["Manufacturer"] + " " + row["Model #"] + " weight", row["Product Name"] + " weight"]
    weight = "NA"
    
    for term in search_terms:
        for url in search(term, stop=1):
            if "bhphotovideo.com" in url or "adorama.com" in url:
                if "bhphotovideo.com" in url:
                    html = requests.get(url).text
                    weight_kg = re.search(r'Product Weight: </span>\n<span>(.*) kg</span>', html)
                elif "adorama.com" in url:
                    html = requests.get(url).text
                    weight_kg = re.search(r'<b>Weight:</b>.*<span class="productData">(.*?)</span>', html)
                
                if weight_kg:
                    weight_kg = float(weight_kg.group(1).replace(",", ""))
                    weight_lb = round(weight_kg * 2.20462, 2)
                    weight = weight_lb
                    
                    break
                
        if weight != "NA":
            break
    
    search_results.loc[i] = [row["Manufacturer"], row["Model #"], row["Product Name"], weight]
    time.sleep(5)

In [ ]:
# Add suffix "_search_results" to the column names of the search_results data frame
search_results = search_results.add_suffix("_search_results")

# Merge the two data frames based on their index
merged_df = pd.concat([df, search_results], axis=1)

In [ ]:
df.to_csv("prduct_page_url.csv", index=False)

In [ ]:
merged_df.to_csv("manual_url.csv", index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download("manual_url.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Miscellaneous Code

In [ ]:
import pandas as pd
from googlesearch import search
import time

df["Model #"] = df["Model #"].astype(str)  # convert integer column to string
# define the search terms
search_terms = df["Manufacturer"] + " " + df["Model #"] + " " + df["Product Name"] + " B&H"
search_terms_adorama = df["Manufacturer"] + " " + df["Model #"] + " " + df["Product Name"] + " Adorama"

# initialize empty lists to store the URLs
urls_bh = []
urls_adorama = []

# search for the URLs on B&H
for search_term in search_terms:
    for url in search(search_term, stop=1):
        urls_bh.append(url)
        break
    else:
        urls_bh.append("NA")
    time.sleep(5)  # add a 5-second delay between each search

# search for the URLs on Adorama
for search_term in search_terms_adorama:
    for url in search(search_term, stop=1):
        urls_adorama.append(url)
        break
    else:
        urls_adorama.append("NA")
    time.sleep(5)  # add a 5-second delay between each search

# add the URLs to the DataFrame
df["B&H URL"] = urls_bh
df["Adorama URL"] = urls_adorama

In [ ]:
# iterate over the rows and add the URLs to the "Product Page URL" column
for index, row in df.iterrows():
# check if the value in the "Product Page URL" column is NaN
    if pd.isna(row["Product Page URL"]):    
        # if a URL was found, add it to the Product Page URL column
        if index < len(urls_bh) and urls_bh[index] != "NA":
            df.at[index, "Product Page URL"] = urls_bh[index]
        elif index < len(urls_adorama) and urls_adorama[index] != "NA":
            df.at[index, "Product Page URL"] = urls_adorama[index]
        else:
            df.at[index, "Product Page URL"] = "NA"

In [1]:
# import pandas as pd
# from googlesearch import search
# import time

# # # create a new empty dataframe
# # search_results = pd.DataFrame(columns=["Manufacturer", "Model #", "Product Name", "B&H URL", "Adorama URL"])

# df["Model #"] = df["Model #"].astype(str)  # convert integer column to string

# # define the search terms
# search_terms = df["Manufacturer"] + " " + df["Model #"] + " " + df["Product Name"] + " B&H"
# search_terms_adorama = df["Manufacturer"] + " " + df["Model #"] + " " + df["Product Name"] + " Adorama"

# # create an empty DataFrame with the correct columns
# search_results = pd.DataFrame(columns=["Manufacturer", "Model #", "Product Name", "B&H URL"])

# # search for the URLs on B&H and store the results in the search_results dataframe
# for i, search_term in enumerate(search_terms):
#     for url in search(search_term, stop=1):
#         search_results.loc[i] = [df.loc[i, "Manufacturer"], df.loc[i, "Model #"], df.loc[i, "Product Name"], url]
#         break
#     else:
#         search_results.loc[i] = [df.loc[i, "Manufacturer"], df.loc[i, "Model #"], df.loc[i, "Product Name"], "NA"]
#     time.sleep(5)


# # search for the URLs on Adorama and add the results to the search_results dataframe
# for i, search_term in enumerate(search_terms_adorama):
#     for url in search(search_term, stop=1):
#         search_results.loc[(i, "Adorama URL")] = url
#         break
#     else:
#         search_results.loc[(i, "Adorama URL")] = "NA"
#     time.sleep(5)